# Crawl Daily Papers and Create a Dataset
This notebook aims to gather the daily papers listed in Hugging Face for the past year and create a dataset that can later be used for analysis.

In [ ]:
!pip install arxiv pypdf scholarly